# PCA method analysis

In [ ]:
include("stuff.jl")

In [ ]:
cube_z_trans = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "z_trans")
cube_pca = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "pca")
cube_pca_online = loadCube(joinpath(cubes_base_dir, "cube_pca_online"))
pca_no_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_no_msc_cube"))
pca_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_msc_cube"))


In [ ]:
cube_base = getCubeData(Cube(data_cube_dir), variable = vars, time = (Date("2001-01-01"), Date("2011-12-31")))

In [ ]:
cumulative_explained_variance = cumsum(cube_pca.prinvars ./ cube_pca.tprinvar)
expl_variance = cube_pca.prinvars ./ cube_pca.tprinvar


In [ ]:
cumulative_explained_variance

In [ ]:
plot(
    0:length(cumulative_explained_variance),
    [0; cumulative_explained_variance], 
    fmt = :png,
    ylab = "Cumulative Explained Variance"
)
scatter!(
    0:length(cumulative_explained_variance),
    [0; cumulative_explained_variance],
    label = ""
)

In [ ]:
R"""
ev <- $expl_variance

par(mar = c(4, 4, 0, 0) + 0.1)
plot(seq_along(ev), ev, type = "c", ylab = "explained fraction of variance", xlab = "PC", bty = "n")
text(seq_along(ev), ev, sprintf("%.2f", ev))
#abline(h = 0.05)
"""

In [ ]:
R"""
par(mar = c(4, 4, 0, 0) + 0.1)
cev <- c(0, $cumulative_explained_variance)
cpc <- seq_along(cev) - 1
plot(cpc, cev, type = "c", 
     xlab = "cumulative principal components", 
     ylab = "cumulative explained variance",
     pch = as.character(seq_along(cev) - 1),
     bty = "n")
text(cpc, cev, sprintf("%0.2f", cev))
"""

In [ ]:
pca_rotation_matrix = cube_pca |> projection

In [ ]:
R"""
rot <- $(rotr90(pca_rotation_matrix))
vars <- $(vars)
n <- dim(rot)[1]
ll <- c(-1, 1) # this line makes the color bar much nicer!!

pal <- brewer.pal(11, "RdBu")
pal <- colorRampPalette(colors = pal)(100)

options(device = function(filename=getOption('rcalljl_filename'),...) png(filename, width = 700, height = 500, ...))

layout(matrix(1:2, ncol = 2), widths = c(1, 0.15))
par(mar = c(4, 14, 2, 0) + 0.1)

image(rot, axes = FALSE, main = "Rotation Matrix", zlim = ll, col = pal, xlab = "PCA")
axis(1, at = seq(0, 1, length.out = n), labels = seq(n), las = 1, cex.axis = 0.9)
axis(2, at = seq(0, 1, length.out = n), labels = rev(vars), las = 2, cex.axis = 1)

par(mar = c(4, 1, 2, 0) + 2.1)

image(x = 1, y = seq(ll[1], ll[2], length.out = length(pal)),
      z = matrix(seq(ll[1], ll[2], length.out = length(pal)), nrow = 1), 
      col = pal, xaxt = "n", xlab = NA, xaxs = "r", ylab = NA, las = 2)
""";

In [ ]:
R"""
pdf($(joinpath(fig_path, "pca.pdf")), width = 8.5, height = 4)
wdth <- c(0.8, 1/1.15, 0.13/1.15)
layout(t(1:3), widths = wdth)

ev <- $expl_variance
ev2 <- ev[1:10]

par(mar = c(4, 4.2, 0, 0) + 0.1)
plot(seq_along(ev2), ev2, xlim = c(1, length(ev)), 
           type = "b", ylab = "Explained Fraction of Variance", xlab = "PC", 
           bty = "n", las = 1, pch = 16, cex.lab = 1.6)
text(seq_along(ev2), ev2, sprintf("%.2f", ev2), adj = c(-0.2, -0.1))
grid.text("(a)", x = unit(0, "npc"), y = unit(1, "npc"),
          just = c("left", "top"))

#abline(h = 0.05)

rot <- $(rotr90(pca_rotation_matrix))
vars <- c(
    "Black Sky Albedo",
    "Evaporation",
    "Evaporative Stress",
    "FAPAR",
    "GPP",
    "LE",
    "NEE",
    "Root Moisture",
    "Sensible Heat",
    "Surface Moisture",
    "Ecosystem Respiration",
    "White Sky Albedo"
)

clamp <- function(x, lims) {
    lo <- lims[1]
    hi <- lims[2]
    res <- x
    res[res < lo] <- lo
    res[res > hi] <- hi
    return(res)
}

#vars <- $(vars)
n <- dim(rot)[1]
max_load <- max(abs(range(rot)))
print(max_load)
ll <- c(-max_load, max_load) # this line makes the color bar much nicer!!

pal <- brewer.pal(11, "RdBu")
pal <- colorRampPalette(colors = pal)(100)

par(mar = c(4, 5, 0, 0) + 0.1)

image(clamp(rot, ll), axes = FALSE, zlim = ll, col = pal, xlab = "PC", useRaster = TRUE, cex.lab = 1.6)
axis(1, at = seq(0, 1, length.out = n), labels = seq(n), las = 1, cex.axis = 1)
axis(2, at = seq(0, 1, length.out = n), labels = rev(vars), las = 2, cex.axis = 1)
grid.text("(b)", x = unit(0.38, "npc"), y = unit(1, "npc"),
          just = c("left", "top"))

par(mar = c(5, 3, 2, 0) + 0.1)

image(x = 1, y = seq(ll[1], ll[2], length.out = length(pal)),
      z = matrix(seq(ll[1], ll[2], length.out = length(pal)), nrow = 1), 
      col = pal, xaxt = "n", xlab = NA, xaxs = "r", ylab = NA, las = 2,
      bty = "n", useRaster = TRUE)

dev.off()
"""

show_pdf(
    "http://127.0.0.1:8333/files/results/fig/pca.pdf",
    width = 900, 
    height = 500
)